In [7]:
from google.colab import drive
drive.mount("/content/drive/", force_remount=True)

Mounted at /content/drive/


###Initialize vgg-19 model

In [0]:
%tensorflow_version 2.x
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.layers import Dense

vgg_19=VGG19(include_top=False,weights='imagenet',input_shape=(256,256,3),pooling='max')
#To continue: https://keras.io/applications/#vgg19

model=Sequential()
for layer in vgg_19.layers:
    layer.trainable=False
    model.add(layer)

model.add(Dense(10,activation='relu'))
model.add(Dense(1,activation='relu'))

model.compile(loss='mean_squared_error',
              optimizer='adam',
              metrics=['accuracy'])
print('Model compiled successfully!')
model.save(filepath="/content/drive/My Drive/Kaggle/TEMP_BEST_MODEL.h5",
               overwrite=True,include_optimizer=True,save_format='h5')
print("Model saved successfully!")

TensorFlow 2.x selected.
Model compiled successfully!
Model saved successfully!


###Training Part

#####Some Constant

In [0]:
#Some Constants
path="/content/drive/My Drive/Kaggle/clips(enhanced)/"
TRAIN_NUM=20000
TEST_NUM=0
BATCH_SIZE=256
STEPS=1+(TRAIN_NUM//BATCH_SIZE)
EPOCHS=5

#####Pickling X and y

In [0]:
#Initialize pkl file
#import pickle
#X_train=[]
#X_test=[]
#with open("/content/drive/My Drive/Kaggle/X_train.pkl",'wb') as f:
#    pickle.dump(X_train,f)
#with open("/content/drive/My Drive/Kaggle/X_test.pkl",'wb') as f:
#     pickle.dump(X_test,f)

#with open("/content/drive/My Drive/Kaggle/_temp",'w') as f:
#    f.write('0')

In [0]:
import pickle
import numpy as np
from PIL import Image
import pandas as pd

#Pickling X
with open("/content/drive/My Drive/Kaggle/X_train.pkl",'rb') as f:
    X_train=pickle.load(f)
with open("/content/drive/My Drive/Kaggle/_temp",'r') as f:
    start=int(f.read())+1

print(f"\nContinue Loading Data {start}")
for m in range(start,25001):
    filepath=path+"clips-"+str(m)+".png"
    X_train.append(np.array(Image.open(filepath)))
    if(m%500==0):
        with open("/content/drive/My Drive/Kaggle/X_train.pkl",'wb') as f:
            pickle.dump(X_train,f)
        with open("/content/drive/My Drive/Kaggle/_temp",'w') as f:
            f.write(str(m))
        print(f"{m} pics stored!")
with open("/content/drive/My Drive/Kaggle/X_train.pkl",'wb') as f:
    pickle.dump(X_train,f)


#Pickling y
y=pd.read_csv("/content/drive/My Drive/Kaggle/master.csv")['clip_count'].to_numpy().reshape((-1,1))
with open("/content/drive/My Drive/Kaggle/y.pkl",'wb') as f:
    pickle.dump(y,f)

####Training

In [4]:
import numpy as np
import pickle
from tqdm import tqdm

with open("/content/drive/My Drive/Kaggle/X.pkl",'rb') as f:
    X=np.array(pickle.load(f))
print("X loaded.")

X=X.astype("bool")

with open("/content/drive/My Drive/Kaggle/y.pkl",'rb') as f:
    y=np.array(pickle.load(f))
print("Y loaded.")

X loaded.
Y loaded.


In [9]:
%tensorflow_version 2.x
import pandas as pd
import numpy as np
from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tqdm import tqdm
from tensorflow.keras.callbacks import ModelCheckpoint
import os
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import load_model

#Load training model
if(os.path.exists("/content/drive/My Drive/Kaggle/TEMP_BEST_MODEL.h5")):
    model=load_model("/content/drive/My Drive/Kaggle/TEMP_BEST_MODEL.h5",compile=True)

    model.compile(loss='mean_squared_error',
              optimizer='adam',
              metrics=['accuracy'])

    print("\nSuccessfully loaded h5 model!\n")
else:
    input("Model load error!")

'''
#Load X and y
print("Loading X and y...")
with open("/content/drive/My Drive/Kaggle/y.pkl",'rb') as f:
    y=np.array(pickle.load(f))
with open("/content/drive/My Drive/Kaggle/X.pkl",'rb') as f:
    X=np.array(pickle.load(f))/255
print("Successfully loaded X and y!")
'''

#Split Train Test
X_train=X[:20000]; X_test=X[20000:]
y_train=y[:20000]; y_test=y[20000:]


#Define Callbacks
checkpoint = ModelCheckpoint("/content/drive/My Drive/Kaggle/TEMP_BEST_MODEL.h5", 
                             monitor='val_loss', verbose=1)
early_stopping = EarlyStopping(monitor='val_loss', patience=5)


#Start Training
print("Training start!")
model.fit(np.array(X_train),np.array(y_train),validation_data=(np.array(X_test),np.array(y_test)),epochs=EPOCHS, verbose=1, callbacks=[early_stopping,checkpoint])

model.save(filepath="/content/drive/My Drive/Kaggle/TEMP_BEST_MODEL.h5",
               overwrite=True,include_optimizer=True,save_format='h5')
print("Trained model saved successfully.")



Successfully loaded h5 model!

Training start!
Train on 20000 samples, validate on 5000 samples
Epoch 1/5
19968/20000 [============================>.] - ETA: 0s - loss: 12.7180 - accuracy: 0.0112
Epoch 00001: saving model to /content/drive/My Drive/Kaggle/TEMP_BEST_MODEL.h5
20000/20000 [==============================] - 310s 16ms/sample - loss: 12.7322 - accuracy: 0.0112 - val_loss: 14.0095 - val_accuracy: 0.0094
Epoch 2/5
19968/20000 [============================>.] - ETA: 0s - loss: 12.7452 - accuracy: 0.0110
Epoch 00002: saving model to /content/drive/My Drive/Kaggle/TEMP_BEST_MODEL.h5
20000/20000 [==============================] - 311s 16ms/sample - loss: 12.7438 - accuracy: 0.0110 - val_loss: 13.8140 - val_accuracy: 0.0100
Epoch 3/5
19968/20000 [============================>.] - ETA: 0s - loss: 12.5988 - accuracy: 0.0111
Epoch 00003: saving model to /content/drive/My Drive/Kaggle/TEMP_BEST_MODEL.h5
20000/20000 [==============================] - 311s 16ms/sample - loss: 12.5919 - 

###Appendix

In [0]:
# Evaluate the generator
for i in train_data_generator():
    for arr in i[0]:
        display(Image.fromarray(arr,'RGB'))
    for val in i[1]:
        print(val)
    input()

In [0]:
from tensorflow.python.client import device_lib

def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return local_device_protos
get_available_gpus()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 2908536609652490470, name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 15280549634419028622
 physical_device_desc: "device: XLA_CPU device", name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 12447679760310874661
 physical_device_desc: "device: XLA_GPU device", name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 15956161332
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 11111954880087899564
 physical_device_desc: "device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0"]

In [0]:
np.array(Image.open("/content/drive/My Drive/Kaggle/clips(enhanced)/clips-1.png")).shape

(256, 256, 3)

In [0]:
def train_data_generator():
    this_batch=[]
    while(1):
        print("NEW EPOCH")
        for m in range(1,100):
            filepath=path+"clips-"+str(m)+".png"
            arr=np.array(Image.open(filepath))
            this_batch.append(arr)
            if(m%256==0):
                yield (np.array(this_batch)/255,y[m-256:m], [None])
                this_batch=[]
        else:
            yield (np.array(this_batch)/255,y[m-len(this_batch):m], [None])
            continue

In [0]:
def train_data_generator():
    this_batch=[]
    while(1):
        print("NEW EPOCH")
        for m in range(1,TRAIN_NUM):
            filepath=path+"clips-"+str(m)+".png"
            this_batch.append(np.array(Image.open(filepath)))
            #print('appended',end=' ')
            #input()
            if(m%BATCH_SIZE==0):
                
                yield (np.array(this_batch)/255,y[m-BATCH_SIZE:m], [None])
                print('BATCH YIELD')
                this_batch=[]
        else:
            yield (np.array(this_batch)/255,y[m-len(this_batch):m], [None])
            continue
checkpoint = ModelCheckpoint("/content/drive/My Drive/Kaggle/TEMP_BEST_MODEL.h5", 
                             monitor='loss', verbose=2)

early_stopping = EarlyStopping(monitor='loss', patience=5)

print("Training start!")
model.fit(train_data_generator(),
                    #validation_data=test_data_generator(),
                    #validation_steps=STEPS,
                    epochs=EPOCHS, verbose=1, 
                    callbacks=[early_stopping,checkpoint]) 

model.save(filepath="/content/drive/My Drive/Kaggle/TEMP_BEST_MODEL.h5",
               overwrite=True,include_optimizer=True,save_format='h5')
print("Trained model saved successfully.")

NEW EPOCH
(256, 256, 256, 3)
(256, 256, 256, 3)
(256, 256, 256, 3)
(256, 256, 256, 3)
(256, 256, 256, 3)


KeyboardInterrupt: ignored